In [1]:
import pandas as pd
import ast
import numpy as np

In [2]:
cols_LOB = ["DateTime","Open","High","Low","Last","Volume","NumTrades","BidVolume","AskVolume","SumBid","SumAsk","BidPrices","BidVolumes","AskPrices","AskVolumes"]

col_list_LOB = ["BidPrices","BidVolumes","AskPrices","AskVolumes"]

In [3]:
def sigmoid(x):
    return 1/(1 + np.exp(-(x.astype(float))))

def string_to_nplist(x):
    if pd.isnull(x):
        return []
    else:
        return np.array(ast.literal_eval(x))

In [4]:
def read_data(file_name, col_names, col_list):
    data = pd.read_csv(file_name, names=col_names, delimiter="|")
    for col in col_list:
        data[col] = data[col].apply(lambda x: string_to_nplist(x))
    data["DateTime"] = pd.to_datetime(data["DateTime"])
    data.set_index(["DateTime"], inplace=True)
    return data

def clean_lob(data, weight_mid_price=0.5, cols_need=["BidPrices","BidVolumes","AskPrices","AskVolumes"], num_level=10):
    lst_valid_samples = []
    mid_prices = []
    for ind, row in data.iterrows():
        if len(row["BidPrices"]) and len(row["AskPrices"]):
            if (row["BidPrices"].shape[0] == num_level) and (row["AskPrices"].shape[0] == num_level):
                lst_valid_samples.append(ind)
                mid_p = weight_mid_price * row["BidPrices"][0] + (1 - weight_mid_price) * row["AskPrices"][0]
                mid_prices.append(mid_p)
    ret_data = pd.DataFrame(index=lst_valid_samples, data=data.loc[lst_valid_samples, cols_need])
    ret_data["Midprice"] = mid_prices
    return ret_data

In [5]:
lob_path = "./"
data_path = lob_path + "LOB_NQU22-CME_2_1_10_10level.lob"

In [7]:
data = pd.read_csv(data_path, names=cols_LOB, delimiter="|")
data.head()

,DateTime,Open,High,Low,Last,Volume,NumTrades,BidVolume,AskVolume,SumBid,SumAsk,BidPrices,BidVolumes,AskPrices,AskVolumes
0,2022-07-07 00:00:00,11897.50,11898.25,11892.75,11893.00,53.0,51.0,34.0,19.0,29.0,20.0,"11892.750000,11892.500000,11892.250000,11892.0...","1,2,2,3,1,4,1,9,3,3,","11893.250000,11893.500000,11893.750000,11894.0...","1,2,3,1,1,2,2,2,3,3,"
1,2022-07-07 00:00:10,11892.25,11892.75,11878.25,11880.00,231.0,211.0,166.0,65.0,22.0,18.0,"11879.750000,11879.500000,11879.250000,11879.0...","1,2,2,3,2,2,3,3,2,2,","11880.250000,11880.500000,11880.750000,11881.0...","1,1,2,2,3,3,1,2,1,2,"
2,2022-07-07 00:00:20,11879.50,11880.50,11876.25,11876.75,85.0,85.0,38.0,47.0,34.0,24.0,"11876.500000,11876.250000,11876.000000,11875.7...","1,1,6,1,3,5,4,7,2,4,","11877.000000,11877.250000,11877.500000,11877.7...","2,4,1,1,3,2,5,1,4,1,"
3,2022-07-07 00:00:30,11876.25,11882.50,11876.00,11880.25,89.0,89.0,36.0,53.0,14.0,19.0,"11880.000000,11879.750000,11879.500000,11879.2...","2,2,1,1,2,1,1,2,1,1,","11880.500000,11880.750000,11881.000000,11881.2...","3,4,1,2,1,1,1,1,4,1,"
4,2022-07-07 00:00:40,11879.50,11882.25,11878.75,11882.25,22.0,22.0,8.0,14.0,14.0,26.0,"11881.500000,11881.250000,11881.000000,11880.7...","1,2,2,1,1,2,1,1,1,2,","11882.500000,11882.750000,11883.000000,11883.2...","5,1,1,4,1,4,2,2,2,4,"


In [16]:
dtaa.head()

str

In [30]:
data.loc[data["BidVolumes"].isnull()]

,DateTime,Open,High,Low,Last,Volume,NumTrades,BidVolume,AskVolume,SumBid,SumAsk,BidPrices,BidVolumes,AskPrices,AskVolumes


In [ ]:
datt groupby by date 
for d in days:
    for i in range(ind_0(d), ind_n(d)-100, step=1):
        sample = X[i: i+100]
        ...